# Fortschrittsanzeige und Datentransformationen

Häufig wirst du in Jupyter-Notebooks mit Daten arbeiten. Manchmal können Operationen *lange dauern* und du möchtest jeweils immer über den aktuellen Status informiert sein.

Sehr oft liegen Daten nicht in exakt der Form vor, in der du sie für die Weiterverarbeitung benötigst. In `pandas` gibt es sehr leistungsfähige Operationen zur *Datentransformation*, die du hier kennenlernen wirst.

## Fortschrittsanzeige

Damit du keine separate Schleife mit langer Laufzeit implementieren musst, simulierst du das einfach mit einem `sleep`, der die Laufzeit der Anweisung in der Schleife auf etwa 10ms erhöht.

In [ ]:
import time
for i in range(1000):
    time.sleep(0.01)

Nach 10s ist der Spuk vorbei, so lange kannst du natürlich warten. Oft dauert es aber deutlich länger und du möchtest wissen, wie weit der Prozess schon fortgeschritten ist.

Dabei hilft dir das Paket `tqdm`. Statt `range` verwendest du einfach `trange`, ansonsten ändert sich nichts. Allerdings sieht das Ergebnis schon etwas anders aus:

In [ ]:
!pip install tqdm

In [ ]:
from tqdm.auto import trange
for i in trange(1000):
    time.sleep(0.01)

Neben dem Fortschritt zeigt `tqdm` auch noch den prozentualen Fertigstellungsgrad, die Iterationen pro Sekunde und die erwartete Restlaufzeit an. Das wirst du oft brauchen, es ist super nützlich.

Es ist natürlich ziemlich praktisch, dass `tqdm` die Funktion `trange` anbietet. Aber sicher möchtest du nicht immer nur mit `range` arbeiten, sondern hast oft auch andere Objekte.

Zum Glück kann `tqdm` auch damit umgehen. Bei bestimmten (iterierbaren) Objekten musst du noch den Parameter `total` eingeben, damit die gesamte Laufzeit abgeschätzt werden kann. Bei `range` ist das nicht notwendig.

`tqdm` kann außerdem auch mit geschachtelten Schleifen umgehen:

In [ ]:
from tqdm.auto import tqdm
for i in tqdm(range(10), desc="first"):
    for j in tqdm(range(100), desc="second"):
        time.sleep(0.01)

Sobald du längere Operationen in Schleifen durchführst, solltest du `tqdm` verwenden. Der zusätzliche Aufwand ist praktisch vernachlässigbar und du erhältst wesentlich mehr Kontrolle über deine Notebooks und den jeweiligen Bearbeitungsstand.

Übrigens funktioniert `tqdm` auch auf der Konsole ziemlich gut und gibt dort (in ASCII-Zeichen) genau die gleichen Informationen aus!

## Datentransformation

Häufig kannst du dir das Format nicht aussuchen, in dem Daten vorliegen. Im Gegenteil, in den allermeisten Fällen ist es schon ein ganz großes Glück, wenn du überhaupt Daten findest und auf die zugreifen kannst.

Einen großen Fundus an Daten bietet dir [Eurostat](https://ec.europa.eu/eurostat), die Statistik-Behörde der Europäischen Union. Natürlich handelt es sich dabei um sehr spezielle Daten, allerdings hast du den großen Vorteil, dass du die Daten frei verwenden darfst und das Herunterladen kein Problem ist. Dafür gibt es nämlich ein Python-Paket `eurostat`:

In [ ]:
!pip install eurostat

Alle Statistikdaten haben bei Eurostat ein Kürzel. In diesem Beispiel beschäftigst du dich mit den *Konsumindikatoren*, diese tragen das Kürzel `ei_bsco_m`. Es handelt sich dabei um sog. *business and consumer surveys*, die monatlich verdichtet sind (alternativ könntest du die auch auf das Quartal aggregiert herunterladen). Eine genauere Erklärung findest du [hier](https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm). 

Das Herunterladen der Daten ist nun sehr einfach:

In [ ]:
import eurostat
df = eurostat.get_data_df("ei_bsco_m")

Betrachte im nächsten Schritt die Daten:

In [ ]:
df

Nun beginnst du mit den Datentransformationen, und zwar zunächst mit den Spaltennamen:

`geo\time` ist eine etwas unglückliche Bezeichnung, hier ist nur das Land gemeint, das kannst du einfach umbenennen:

In [ ]:
df = df.rename(columns={"geo\\TIME_PERIOD": "country"})

Ebenso etwas merkwürdig sind die Monatsnamen in Form von `2021M04` etc., die du auch besser durch ein `datetime`-Objekt in Python ersetzt:

In [ ]:
from datetime import datetime
df.columns = [datetime.strptime(f.split("-")[0] + "-" + f.split("-")[1] + "-01", "%Y-%m-%d")
                if f.startswith("20") or f.startswith("19") else f for f in df.columns]

Betrachte das Ergebnis:

In [ ]:
df

Das sieht schon viel besser aus, aber es sind ziemlich viele Zeilen, weil sehr viele Indikatoren enthalten sind.

Nun *filterst* du die Daten und nutzt nur den Indikator `BS_CSMCI` (Konsumentenvertrauen) und zwar dessen *saison-justierte Variante*:

In [ ]:
bs = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA")]
bs

Das ist schon viel übersichtlicher, konzentriere dich nun zunächst auf Deutschland:

In [ ]:
bs[bs["country"] == "DE"]

Das Ergebnis sieht schon gut aus, allerdings benötigst du nur die Spalten mit den Monaten, hier interessiert du dich außerdem nur für Daten ab 2010. Wenn du eine `list` in `pandas` als Index verwendest, werden nur die Spalten benutzt:

In [ ]:
bs_de = bs[bs["country"] == "DE"][[c for c in bs.columns 
                     if (isinstance(c, datetime) and c.year>=2010)]]
bs_de

Genau das wolltest du, jetzt kannst du es gleich plotten:

In [ ]:
bs_de.plot()

Das hat leider nicht wie gewünscht funktioniert, weil der Plot die Werte in *vielen Zeilen* benötigt. Spalten und Zeilen vertauschen nennt sich *transponieren* und das kannst du in `pandas` einfach durch die Operation `.T` erreichen:

In [ ]:
bs_de.T.plot()

Genau dieses Ergebnis wolltest du erreichen. Die *Corona-Krise* kannst du im Konsumindikator gut erkennen.

Willst du alle Konsumindikatoren über die Länder zu einem Zeitpunkt (z.B. Juni 2021) hinweg vergleichen, bietet sich dafür eine Tabelle an. Die Daten hast du allerdings nicht in der richtigen Form zur Verfügung, weil sowohl  Konsumindikatoren als auch Länder in den Zeilen dargestellt sind. Du möchtest aber die Konsumindikatoren in den Spalten und die Länder in den Zeilen haben.

Mit `pandas` geht das ganz einfach, denn dazu gibt es die Funktion `pivot`, der du einfach die Namen der Zeilen (`index`) und die Namen der Spalten (`columns`) übergibst. Zusätzlich musst du der Funktion mitteilen, welche Werte (aus den ursprünglichen Spalten) sie verwenden soll:

In [ ]:
indic_country = df[df["s_adj"] == "SA"].pivot(index="country", columns="indic", 
                                              values=datetime(2022, 11, 1))
indic_country

Eine etwas übersichtlichere Darstellung erreichst du mit einer sog. *Heatmap*:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))
sns.heatmap(indic_country, annot=True)

Hier siehst du auf den ersten Blick, dass Rumänien keine Werte geliefert hat und der Indikator `BS_SV_PR` für die gesamte EU bzw. den Euro-Raum nicht berechnet wurde.